In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import keras

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, Dropout

from numpy.random import seed
from tensorflow import set_random_seed
from sklearn.metrics import roc_auc_score

In [0]:
test = pd.read_csv("https://raw.githubusercontent.com/dk1000/titanic/master/titanic_test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/dk1000/titanic/master/titanic_train.csv")
etykieta = pd.read_csv("https://raw.githubusercontent.com/dk1000/titanic/master/gender_baseline.csv")

In [0]:
#test.head()

In [0]:
# convert to cateogry dtype
test['sex'] = test['sex'].astype('category')
train['sex'] = train['sex'].astype('category')

# convert to category codes
test['sex'] = test['sex'].cat.codes
train['sex'] = train['sex'].cat.codes

In [0]:
# subset all categorical variables which need to be encoded
categorical = ['embarked']

for var in categorical:
    test = pd.concat([test, 
                    pd.get_dummies(test[var], prefix=var)], axis=1)
    del test[var]
    
    
categorical = ['embarked']

for var in categorical:
    train = pd.concat([train, 
                    pd.get_dummies(train[var], prefix=var)], axis=1)
    del train[var]

In [0]:
#train.head(2)

In [0]:
train = train.iloc[:, [1, 3, 4, 5, 6, 8, 14, 15, 16, 13]]
test = test.iloc[:, [1, 3, 4, 5, 6, 8, 13, 14, 15]]

In [0]:
#train.head(2)

In [0]:
df1 = pd.concat([test, etykieta], axis=1, sort=True)
df1 = df1.dropna()
train = train.dropna()

In [0]:
X_train = train.iloc[:, [0,1,2,3,4,5,6,7,8,]].values
Y_train = train.iloc[:, 9].values

X_test = df1.iloc[:, [0,1,2,3,4,5,6,7,8]].values
Y_test = df1.iloc[:, 10].values

In [0]:
model = Sequential()
model.add(Dense(units=200, activation='sigmoid'))
model.add(Dense(units=100, activation='sigmoid'))
model.add(Dense(units=30, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss = keras.losses.binary_crossentropy,
              optimizer = keras.optimizers.Adam(),
              metrics = ['accuracy'])

In [499]:
batch_size = 30
epochs = 10

model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test))

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 675 samples, validate on 370 samples
Epoch 1/10
675/675 [==============================] - 7s 10ms/step - loss: 0.6653 - acc: 0.5763 - val_loss: 0.6406 - val_acc: 0.6162
Epoch 2/10
675/675 [==============================] - 0s 186us/step - loss: 0.6245 - acc: 0.6578 - val_loss: 0.6266 - val_acc: 0.6622
Epoch 3/10
675/675 [==============================] - 0s 196us/step - loss: 0.6102 - acc: 0.6696 - val_loss: 0.6273 - val_acc: 0.6541
Epoch 4/10
675/675 [==============================] - 0s 175us/step - loss: 0.6097 - acc: 0.6726 - val_loss: 0.6260 - val_acc: 0.6622
Epoch 5/10
675/675 [==============================] - 0s 184us/step - loss: 0.6041 - acc: 0.6741 - val_loss: 0.6238 - val_acc: 0.6568
Epoch 6/10
675/675 [==============================] - 0s 181us/step - loss: 0.6039 - acc: 0.6815 - val_loss: 0.6196 - val_acc: 0.6595
Epoch 7/10
675/675 [==============================] - 0s 173us/step - loss: 0.5993 - acc: 0.6800 - val_loss: 0.6192 - val_acc: 0.6622
Epoch 8/10
675/67

In [500]:
y_pred = model.predict_proba(X_test)
print("AUC", roc_auc_score(Y_test, y_pred))

AUC 0.7082715591796392
